# I wouldn't run this notebook and just load the models but use this to see how I cleaned my data  Each model notebook loads the complete pickled dataframe anyway

after initial bring good features in

logistic regression
random forest (single decision tree)
(no SVM due to large dataset)

ROC curves
precision, recall
f1 score
type 1 error and type 2 error 

cost benefit analysis?  

Would be nice to have some landfall yes or no feature  
Need to impute damage_property and deaths by category mean  
Need to patsy disaster category type

In [1]:
from __future__ import print_function, division
import requests
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import ElasticNetCV
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from datetime import datetime
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [2]:
import pickle
with open("wind_df.pkl", 'rb') as picklefile: 
    wind_df = pickle.load(picklefile)

In [3]:
with open("climate_df.pkl", 'rb') as picklefile: 
    climate_df = pickle.load(picklefile)

In [5]:
wind_df.EVENT_TYPE.unique()
# wind_df.info(verbose = True, null_counts = True)

array(['Thunderstorm Wind', 'THUNDERSTORM WINDS/FLOODING',
       'THUNDERSTORM WINDS/FLASH FLOOD', 'THUNDERSTORM WINDS LIGHTNING',
       'THUNDERSTORM WIND/ TREES', 'THUNDERSTORM WIND/ TREE',
       'THUNDERSTORM WINDS FUNNEL CLOU', 'THUNDERSTORM WINDS/HEAVY RAIN',
       'THUNDERSTORM WINDS HEAVY RAIN', 'THUNDERSTORM WINDS/ FLOOD',
       'High Wind', 'Strong Wind', 'Tropical Storm',
       'Hurricane (Typhoon)', 'Marine High Wind',
       'Marine Thunderstorm Wind', 'Heavy Wind', 'Marine Strong Wind',
       'Tropical Depression', 'Hurricane', 'Marine Tropical Storm',
       'Marine Hurricane/Typhoon', 'Marine Tropical Depression'],
      dtype=object)

In [6]:
# data observation hurricane data no wind speed
# data observation hurricane(typhoon) data lower wind speed than expected
# thunderstorm wind is highest speed for some reason

drop_wind_df = wind_df.drop(columns = ['YEAR', 'MONTH_NAME', 'FATALITY_ID', 'FAT_DAY', 'FAT_TIME'])
drop_wind_df.EVENT_TYPE.unique()
# drop_wind_df.sort_values(by = 'MAGNITUDE', ascending = False).head(10)
# drop_wind_df[drop_wind_df.EVENT_TYPE == 'Hurricane'].head(100)

array(['Thunderstorm Wind', 'THUNDERSTORM WINDS/FLOODING',
       'THUNDERSTORM WINDS/FLASH FLOOD', 'THUNDERSTORM WINDS LIGHTNING',
       'THUNDERSTORM WIND/ TREES', 'THUNDERSTORM WIND/ TREE',
       'THUNDERSTORM WINDS FUNNEL CLOU', 'THUNDERSTORM WINDS/HEAVY RAIN',
       'THUNDERSTORM WINDS HEAVY RAIN', 'THUNDERSTORM WINDS/ FLOOD',
       'High Wind', 'Strong Wind', 'Tropical Storm',
       'Hurricane (Typhoon)', 'Marine High Wind',
       'Marine Thunderstorm Wind', 'Heavy Wind', 'Marine Strong Wind',
       'Tropical Depression', 'Hurricane', 'Marine Tropical Storm',
       'Marine Hurricane/Typhoon', 'Marine Tropical Depression'],
      dtype=object)

In [7]:
# further minimal manipulations for mvp 
from copy import deepcopy
mvp_wind_df = deepcopy(drop_wind_df)
# mvp_wind_df = drop_wind_df[drop_wind_df.MAGNITUDE != 0]
mvp_wind_df.loc[:, 'BEGIN_YEARMONTH'] = mvp_wind_df['BEGIN_YEARMONTH'].apply(lambda x: str(int(x)))
mvp_wind_df.loc[:, 'BEGIN_YEARMONTH'] = mvp_wind_df['BEGIN_YEARMONTH'].apply(lambda x: datetime.strptime(x,'%Y%m').date())
mvp_wind_df.drop(columns = ['FAT_YEARMONTH', 'FATALITY_AGE', 'FATALITY_SEX', 'BEGIN_TIME', 'BEGIN_DAY', 'DAMAGE_CROPS'], inplace = True)
mvp_wind_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568137 entries, 1867 to 1782691
Data columns (total 15 columns):
BEGIN_YEARMONTH      568137 non-null object
EVENT_ID             568137 non-null int64
STATE                568137 non-null object
EVENT_TYPE           568137 non-null object
INJURIES_DIRECT      568137 non-null float64
INJURIES_INDIRECT    568137 non-null float64
DEATHS_DIRECT        568137 non-null float64
DEATHS_INDIRECT      568137 non-null float64
DAMAGE_PROPERTY      428893 non-null object
MAGNITUDE            518682 non-null float64
MAGNITUDE_TYPE       370838 non-null object
BEGIN_LAT            447389 non-null float64
BEGIN_LON            447389 non-null float64
END_LAT              356463 non-null float64
END_LON              356464 non-null float64
dtypes: float64(9), int64(1), object(5)
memory usage: 69.4+ MB


In [8]:
# attempt to weed out all thunderstorm entries, decided to keep them in due to limited samples otherwise
import re
# def thunderstorm(string):
#     pattern = re.compile(r'[Tt][Hh][Uu][Nn][Dd][Ee][Rr][Ss][Tt][Oo][Rr][Mm]')
#     if re.search(pattern, string):
#         return True
#     return False

# thunderstorm('thudersTORM (sdlfkj)')

In [9]:
mvp_wind_df.sort_values(by = 'DEATHS_DIRECT', ascending = False).head()
mvp_wind_df.loc[mvp_wind_df.EVENT_TYPE == 'Hurricane (Typhoon)'].sort_values(by = 'MAGNITUDE', ascending = False).head(100)
# mvp_wind_df['THUNDERSTORM'] = mvp_wind_df.EVENT_TYPE.apply(thunderstorm)
mvp_wind_df[mvp_wind_df.EVENT_TYPE.str.contains(r'[Tt][Hh][Uu][Nn][Dd][Ee][Rr][Ss][Tt][Oo][Rr][Mm]')].sort_values(by = 'DAMAGE_PROPERTY', ascending = False).head()
# mvp_wind_df[mvp_wind_df.EVENT_TYPE.str.contains(r'^(?!.*Thunderstorm).*$')].head()
# mvp_wind_df.drop(columns = 'THUNDERSTORM', inplace = True)
mvp_wind_df.head()

,BEGIN_YEARMONTH,EVENT_ID,STATE,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,MAGNITUDE,MAGNITUDE_TYPE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON
1867,1955-06-01,9976778,ALABAMA,Thunderstorm Wind,0.0,0.0,0.0,0.0,0,0.0,NaN,33.00,-86.78,NaN,NaN
1869,1955-07-01,9976780,ALABAMA,Thunderstorm Wind,0.0,0.0,0.0,0.0,0,52.0,NaN,33.58,-86.78,NaN,NaN
1874,1955-10-01,9976786,ALABAMA,Thunderstorm Wind,0.0,0.0,0.0,0.0,0,0.0,NaN,33.58,-86.78,NaN,NaN
1881,1955-03-01,10005140,INDIANA,Thunderstorm Wind,0.0,0.0,0.0,0.0,0,0.0,NaN,41.00,-85.18,NaN,NaN
1883,1955-05-01,10000327,GEORGIA,Thunderstorm Wind,0.0,0.0,0.0,0.0,0,0.0,NaN,33.00,-85.00,NaN,NaN


In [10]:
# turns out dropping the thunderstorm would yield an extremely sparse dataframe, it did tell me that i need to impute some values for hurricanes
test_mvp_wind_df = mvp_wind_df[mvp_wind_df.EVENT_TYPE.str.contains(r'^(?!.*Thunderstorm).*$')]
test_mvp_wind_df = test_mvp_wind_df[test_mvp_wind_df.EVENT_TYPE.str.contains(r'^(?!.*THUNDERSTORM).*$')]
test_mvp_wind_df[test_mvp_wind_df.EVENT_TYPE.isin(['Tropical Storm',
       'Hurricane (Typhoon)', 'Tropical Depression', 'Hurricane',
       'Marine Tropical Storm', 'Marine Hurricane/Typhoon',
       'Marine Tropical Depression'])].STATE.unique()
# test_mvp_wind_df.EVENT_TYPE.unique()
test_mvp_wind_df[test_mvp_wind_df.EVENT_TYPE.isin(['Tropical Storm',
       'Hurricane (Typhoon)', 'Tropical Depression', 'Hurricane',
       'Marine Tropical Storm', 'Marine Hurricane/Typhoon',
       'Marine Tropical Depression'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8247 entries, 240281 to 1767446
Data columns (total 15 columns):
BEGIN_YEARMONTH      8247 non-null object
EVENT_ID             8247 non-null int64
STATE                8247 non-null object
EVENT_TYPE           8247 non-null object
INJURIES_DIRECT      8247 non-null float64
INJURIES_INDIRECT    8247 non-null float64
DEATHS_DIRECT        8247 non-null float64
DEATHS_INDIRECT      8247 non-null float64
DAMAGE_PROPERTY      5454 non-null object
MAGNITUDE            44 non-null float64
MAGNITUDE_TYPE       0 non-null object
BEGIN_LAT            0 non-null float64
BEGIN_LON            0 non-null float64
END_LAT              0 non-null float64
END_LON              0 non-null float64
dtypes: float64(9), int64(1), object(5)
memory usage: 1.0+ MB


In [11]:
# mvp_wind_df.loc[mvp_wind_df['DAMAGE_PROPERTY'] == '2H','DAMAGE_PROPERTY'] = '2K'
# mvp_wind_df.loc[mvp_wind_df['DAMAGE_PROPERTY'] == '2h','DAMAGE_PROPERTY'] = '2K'

In [12]:
def convert_to_int(string):
    if 'K' or 'H' or 'h' in string:
        try:
            return float(string[:-1].replace(' ',''))*1000
        except:
            return None
    elif 'M' in string:
        try:
            return float(string[:-1].replace(' ',''))*1000000
        except:
            return None
    elif 'B' in string:
        try:
            return float(string[:-1].replace(' ',''))*1000000000
        except:
            return None
    else:
        return float(string)

In [13]:
from time import time
start = time()
test_mvp_wind_df = deepcopy(mvp_wind_df)
test_mvp_wind_df = test_mvp_wind_df.DAMAGE_PROPERTY.apply(convert_to_int)
concat_mvp_wind_df = mvp_wind_df.drop(columns=['DAMAGE_PROPERTY'])
mvp_wind_df = pd.concat([concat_mvp_wind_df, test_mvp_wind_df], axis = 1)
mvp_wind_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568137 entries, 1867 to 1782691
Data columns (total 15 columns):
BEGIN_YEARMONTH      568137 non-null object
EVENT_ID             568137 non-null int64
STATE                568137 non-null object
EVENT_TYPE           568137 non-null object
INJURIES_DIRECT      568137 non-null float64
INJURIES_INDIRECT    568137 non-null float64
DEATHS_DIRECT        568137 non-null float64
DEATHS_INDIRECT      568137 non-null float64
MAGNITUDE            518682 non-null float64
MAGNITUDE_TYPE       370838 non-null object
BEGIN_LAT            447389 non-null float64
BEGIN_LON            447389 non-null float64
END_LAT              356463 non-null float64
END_LON              356464 non-null float64
DAMAGE_PROPERTY      322855 non-null float64
dtypes: float64(10), int64(1), object(4)
memory usage: 69.4+ MB


In [14]:
# mvp_wind_df[mvp_wind_df.MAGNITUDE == 0].groupby(by = 'EVENT_TYPE').count()
# mvp_wind_df.groupby(by = 'EVENT_TYPE')[['MAGNITUDE','DAMAGE_PROPERTY','BEGIN_LAT','BEGIN_LON']].agg(['sum','count','mean'])
mvp_wind_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568137 entries, 1867 to 1782691
Data columns (total 15 columns):
BEGIN_YEARMONTH      568137 non-null object
EVENT_ID             568137 non-null int64
STATE                568137 non-null object
EVENT_TYPE           568137 non-null object
INJURIES_DIRECT      568137 non-null float64
INJURIES_INDIRECT    568137 non-null float64
DEATHS_DIRECT        568137 non-null float64
DEATHS_INDIRECT      568137 non-null float64
MAGNITUDE            518682 non-null float64
MAGNITUDE_TYPE       370838 non-null object
BEGIN_LAT            447389 non-null float64
BEGIN_LON            447389 non-null float64
END_LAT              356463 non-null float64
END_LON              356464 non-null float64
DAMAGE_PROPERTY      322855 non-null float64
dtypes: float64(10), int64(1), object(4)
memory usage: 69.4+ MB


In [15]:
mvp_wind_df = mvp_wind_df[mvp_wind_df.EVENT_TYPE != 'Heavy Wind']
mvp_wind_df.loc[mvp_wind_df.EVENT_TYPE == 'Hurricane (Typhoon)', 'EVENT_TYPE'] = 'Hurricane'
mvp_wind_df = mvp_wind_df[mvp_wind_df.EVENT_TYPE.str.contains(r'^(?!.*THUNDERSTORM).*$')]
mvp_wind_df.EVENT_TYPE.unique()

array(['Thunderstorm Wind', 'High Wind', 'Strong Wind', 'Tropical Storm',
       'Hurricane', 'Marine High Wind', 'Marine Thunderstorm Wind',
       'Marine Strong Wind', 'Tropical Depression',
       'Marine Tropical Storm', 'Marine Hurricane/Typhoon',
       'Marine Tropical Depression'], dtype=object)

In [16]:
# impute all things needed to be imputed
avg_speeds = dict(mvp_wind_df.groupby(by = 'EVENT_TYPE').mean()['MAGNITUDE'])
avg_speeds['Marine Hurricane/Typhoon'] = avg_speeds['Hurricane']
avg_speeds['Marine Tropical Depression'] = 0
avg_speeds['Marine Tropical Storm'] = avg_speeds['Tropical Storm']
avg_speeds['Tropical Depression'] = 0
avg_damage = dict(mvp_wind_df.groupby(by = 'EVENT_TYPE').mean()['DAMAGE_PROPERTY'])
avg_lat = mvp_wind_df.BEGIN_LAT.mean()
avg_lon = mvp_wind_df.BEGIN_LON.mean()

avg_damage

{'High Wind': 19459.206944750975,
 'Hurricane': 70885.64417177915,
 'Marine High Wind': 1329.2564102564102,
 'Marine Hurricane/Typhoon': 103.44827586206897,
 'Marine Strong Wind': 4443.115942028986,
 'Marine Thunderstorm Wind': 263.4190036518983,
 'Marine Tropical Depression': 0.0,
 'Marine Tropical Storm': 0.0,
 'Strong Wind': 13416.28300303402,
 'Thunderstorm Wind': 16578.44222698038,
 'Tropical Depression': 106625.95238095238,
 'Tropical Storm': 65063.12433955618}

In [17]:
# now plug into magnitude Nans
from copy import deepcopy
test_mvp_wind_df = deepcopy(mvp_wind_df)
for event_type in avg_speeds:
    test_mvp_wind_df.loc[((test_mvp_wind_df.MAGNITUDE.isnull()) & (test_mvp_wind_df.EVENT_TYPE == event_type)), 'MAGNITUDE'] = avg_speeds[event_type]
    print(event_type)
for event_type in avg_damage:
    test_mvp_wind_df.loc[((test_mvp_wind_df.DAMAGE_PROPERTY.isnull()) & (test_mvp_wind_df.EVENT_TYPE == event_type)), 'DAMAGE_PROPERTY'] = avg_damage[event_type]
    print(event_type)
test_mvp_wind_df.loc[test_mvp_wind_df.BEGIN_LAT.isnull(), 'BEGIN_LAT'] = avg_lat
test_mvp_wind_df.loc[test_mvp_wind_df.BEGIN_LON.isnull(), 'BEGIN_LON'] = avg_lon                    

High Wind


KeyboardInterrupt: 

In [18]:
mvp_wind_df = test_mvp_wind_df

In [22]:
mvp_wind_df = mvp_wind_df.drop(columns = ['MAGNITUDE_TYPE', 'END_LAT', 'END_LON'])
mvp_wind_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568119 entries, 1867 to 1782691
Data columns (total 12 columns):
BEGIN_YEARMONTH      568119 non-null object
EVENT_ID             568119 non-null int64
STATE                568119 non-null object
EVENT_TYPE           568119 non-null object
INJURIES_DIRECT      568119 non-null float64
INJURIES_INDIRECT    568119 non-null float64
DEATHS_DIRECT        568119 non-null float64
DEATHS_INDIRECT      568119 non-null float64
MAGNITUDE            568119 non-null float64
BEGIN_LAT            568119 non-null float64
BEGIN_LON            568119 non-null float64
DAMAGE_PROPERTY      568119 non-null float64
dtypes: float64(8), int64(1), object(3)
memory usage: 56.3+ MB


In [23]:
import pickle
with open('mvp_wind_df' + '.pkl', 'wb') as picklefile:
    pickle.dump(mvp_wind_df, picklefile)

In [29]:
climate_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1380 entries, 0 to 1379
Data columns (total 3 columns):
tas                1380 non-null float64
BEGIN_YEARMONTH    1380 non-null object
pr                 1380 non-null float64
dtypes: float64(2), object(1)
memory usage: 43.1+ KB


In [39]:
combined_df = pd.merge(climate_df,mvp_wind_df, how = 'inner', on = 'BEGIN_YEARMONTH')
combined_df.rename(columns = {'tas': 'Temperature', 'pr': 'Precipitation'}, inplace = True)

In [40]:
import pickle
with open('combined_df' + '.pkl', 'wb') as picklefile:
    pickle.dump(combined_df, picklefile)

In [36]:
import patsy
dummified_df = patsy.dmatrix('STATE', data = mvp_wind_df, return_type = 'dataframe')
dummified_df.drop(columns = ['Intercept'], inplace = True)
dummified_mvp_wind_df = pd.concat([mvp_wind_df, dummified_df], axis=1)
dummified_mvp_wind_df.drop(columns = ['STATE'], inplace = True)
dummified_mvp_wind_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568119 entries, 1867 to 1782691
Data columns (total 78 columns):
BEGIN_YEARMONTH                  568119 non-null object
EVENT_ID                         568119 non-null int64
EVENT_TYPE                       568119 non-null object
INJURIES_DIRECT                  568119 non-null float64
INJURIES_INDIRECT                568119 non-null float64
DEATHS_DIRECT                    568119 non-null float64
DEATHS_INDIRECT                  568119 non-null float64
MAGNITUDE                        568119 non-null float64
BEGIN_LAT                        568119 non-null float64
BEGIN_LON                        568119 non-null float64
DAMAGE_PROPERTY                  568119 non-null float64
STATE[T.ALASKA]                  568119 non-null float64
STATE[T.AMERICAN SAMOA]          568119 non-null float64
STATE[T.ARIZONA]                 568119 non-null float64
STATE[T.ARKANSAS]                568119 non-null float64
STATE[T.ATLANTIC NORTH]          5

In [37]:
dummified_df = patsy.dmatrix('EVENT_TYPE', data = dummified_mvp_wind_df, return_type = 'dataframe')
dummified_df.drop(columns = ['Intercept'], inplace = True)
dummified_mvp_wind_df = pd.concat([dummified_mvp_wind_df, dummified_df], axis=1)
dummified_mvp_wind_df.drop(columns = ['EVENT_TYPE'], inplace = True)
dummified_mvp_wind_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568119 entries, 1867 to 1782691
Data columns (total 88 columns):
BEGIN_YEARMONTH                             568119 non-null object
EVENT_ID                                    568119 non-null int64
INJURIES_DIRECT                             568119 non-null float64
INJURIES_INDIRECT                           568119 non-null float64
DEATHS_DIRECT                               568119 non-null float64
DEATHS_INDIRECT                             568119 non-null float64
MAGNITUDE                                   568119 non-null float64
BEGIN_LAT                                   568119 non-null float64
BEGIN_LON                                   568119 non-null float64
DAMAGE_PROPERTY                             568119 non-null float64
STATE[T.ALASKA]                             568119 non-null float64
STATE[T.AMERICAN SAMOA]                     568119 non-null float64
STATE[T.ARIZONA]                            568119 non-null float64
STAT

In [42]:
combined_dummified_df = pd.merge(climate_df,dummified_mvp_wind_df, how = 'inner', on = 'BEGIN_YEARMONTH')
combined_dummified_df.rename(columns = {'tas': 'Temperature', 'pr': 'Precipitation'}, inplace = True)

import pickle
with open('combined_dummified_df' + '.pkl', 'wb') as picklefile:
    pickle.dump(combined_dummified_df, picklefile)